In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.rendered_html { font-size: 18px; }</style>"))

First initial libraries. Machine learning has been left out for now.

Additionally, the datasets are much too large for my laptops memory, so I will load truncated versions. I will also keep a commented-out cell which attempts to either load the whole dataset, or use a SQL server for it.

In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import nltk
import datetime
import math
#from sqlalchemy import create_engine
#nltk.download()

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

The first cell here will upload smaller sections of the datasets. We will go in a roundabout way in order to upload datafiles to github. Which will be commented out once used

In [3]:
#csv_2016 = pd.read_csv('properties_2016.csv', nrows = 30000)
#csv_2017 = pd.read_csv('properties_2017.csv', nrows = 30000)
#csv_2016.to_csv('2016.csv')
#csv_2017.to_csv('2017.csv')

In [4]:

raw_2016 = pd.read_csv('2016.csv')
raw_2017 = pd.read_csv('2017.csv')
vals_2016 = pd.read_csv('train_2016_v2.csv')
vals_2017 = pd.read_csv('train_2017.csv')


The next cell will load the entire datasets, and I will likely not ever be able to use this cell.

In [5]:
#raw_2016 = pd.read_csv('properties_2016.csv', low_memory = False)
#raw_2017 = pd.read_csv('properties_2017.csv', low_memory = False)
#vals_2016 = pd.read_csv('train_2016_v2.csv')
#vals_2017 = pd.read_csv('train_2017.csv')

Finally a third possible solution that lets me use SQL to pull in large datasets after I have figured out how to trim the dataset.

This still has big memory issues, so it may not end up being used either. I would also like to know if there is a way to minimize a cell as this the SQL cell takes up a lot of room.

In [6]:
'''
vals_2016 = pd.read_csv('train_2016_v2.csv')
vals_2017 = pd.read_csv('train_2017.csv')

csv_database = create_engine('sqlite:///csv_database.db')

# Next we load a the 2016 dataset into a SQL engine table chunk by chunk
chunksize = 10000
i_1 = 0
j_1 = 1
for df in pd.read_csv('properties_2016.csv', chunksize=chunksize, iterator=True):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j_1
      i_1+=1
      df.to_sql('table_2016', csv_database, if_exists='append')
      j = df.index[-1] + 1
#now we do the same for 2017 into another table
i_2 = 0
j_2 = 1
for df in pd.read_csv('properties_2017.csv', chunksize=chunksize, iterator=True):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j_2
      i_2+=1
      df.to_sql('table_2017', csv_database, if_exists='append')
      j = df.index[-1] + 1
        
df = pd.read_sql_query('SELECT COl1 table_2017', csv_database)
df.head()
'''
print()

Before we can merge the dataframes with their vals we need to see how long each are. We note that there are far more in the vals dataframe, but since we did not truncate those csv files but did for the raw files, we do not know if vals is complete.

In [7]:
len(raw_2016),len(vals_2016)

(30000, 90275)

Now we will merge the raw df with the vals df. However, we note that the df's when merged on 'inner' are MUCH smaller than the starting ones, meaning that the train csv's were smaller than the raw csv's. However, since this notebook is primary concerned with munging and cleaning, we will proceed anyway, hoping for a solution where we can deal with the larger datasets.

Another important observation is that the mergeing process gained us two extra rows so we no longer have unique entries in parcelid. However, the vals dataframe (which was not truncated) has several repeated parcelid's meaning that the original dataset will have likely contained a great number of repeated parcelids. As such we will leave them be.

In [9]:
df2016 = pd.merge(raw_2016, vals_2016, how='left', left_on='parcelid', right_on='parcelid')
df2017 = pd.merge(raw_2017, vals_2017, how='left', left_on='parcelid', right_on='parcelid')
print(len(pd.merge(raw_2016, vals_2016, how='inner', left_on='parcelid', right_on='parcelid')))
print(len(df2016['parcelid']),len(df2016['parcelid'].unique()))
print(len(df2017['parcelid']),len(df2017['parcelid'].unique()))
print(len(vals_2016['parcelid']), len(vals_2016['parcelid'].unique()))

896
30002 30000
30003 30000
90275 90150


Now lets get some information on the structure of these dataframes as well as what types of data are inside it. The biggest observations we can make here is that there are many empty columns, likely resulting from the extreme truncation. As a result, we will need merge with how='left'.

In [10]:
pd.merge(raw_2016, vals_2016, how='inner', left_on='parcelid', right_on='parcelid').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896 entries, 0 to 895
Data columns (total 61 columns):
Unnamed: 0                      896 non-null int64
parcelid                        896 non-null int64
airconditioningtypeid           276 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     896 non-null float64
bedroomcnt                      896 non-null float64
buildingclasstypeid             0 non-null float64
buildingqualitytypeid           551 non-null float64
calculatedbathnbr               884 non-null float64
decktypeid                      7 non-null float64
finishedfloor1squarefeet        67 non-null float64
calculatedfinishedsquarefeet    890 non-null float64
finishedsquarefeet12            851 non-null float64
finishedsquarefeet13            0 non-null float64
finishedsquarefeet15            35 non-null float64
finishedsquarefeet50            67 non-null float64
finishedsqu

In [11]:
df2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30002 entries, 0 to 30001
Data columns (total 61 columns):
Unnamed: 0                      30002 non-null int64
parcelid                        30002 non-null int64
airconditioningtypeid           8087 non-null float64
architecturalstyletypeid        70 non-null float64
basementsqft                    16 non-null float64
bathroomcnt                     30002 non-null float64
bedroomcnt                      30002 non-null float64
buildingclasstypeid             179 non-null float64
buildingqualitytypeid           19323 non-null float64
calculatedbathnbr               28530 non-null float64
decktypeid                      172 non-null float64
finishedfloor1squarefeet        2040 non-null float64
calculatedfinishedsquarefeet    29474 non-null float64
finishedsquarefeet12            27217 non-null float64
finishedsquarefeet13            90 non-null float64
finishedsquarefeet15            1953 non-null float64
finishedsquarefeet50           

Now we no longer have any empty columns. Unfortunately some are still almost empty. One goal to look into later will be to find a way to read a csv with n randomly selected rows to get a potentially more representative sample of the original dataset.

We want to combine the 2016 and 2017 datasets and now we note that we have a great deal of repeated parcelid entries, so this new dataframe perhaps more closely resembles the larger datasets.

In [12]:
combined_df = pd.concat([df2016,df2017]).copy()
#combined_df.set_index('parcelid', inplace=True)
print(len(combined_df['parcelid'].unique()),len(combined_df['parcelid']))
combined_df.info()

30124 60005
<class 'pandas.core.frame.DataFrame'>
Int64Index: 60005 entries, 0 to 30002
Data columns (total 61 columns):
Unnamed: 0                      60005 non-null int64
parcelid                        60005 non-null int64
airconditioningtypeid           16222 non-null float64
architecturalstyletypeid        140 non-null float64
basementsqft                    31 non-null float64
bathroomcnt                     60005 non-null float64
bedroomcnt                      60005 non-null float64
buildingclasstypeid             361 non-null float64
buildingqualitytypeid           38701 non-null float64
calculatedbathnbr               57178 non-null float64
decktypeid                      350 non-null float64
finishedfloor1squarefeet        4084 non-null float64
calculatedfinishedsquarefeet    59014 non-null float64
finishedsquarefeet12            54514 non-null float64
finishedsquarefeet13            174 non-null float64
finishedsquarefeet15            3904 non-null float64
finishedsquarefe

Nearly all our data is numberical and thus requires little cleaning in that respect. But we will take a closer look at the data fields which are non-numberical in case they require cleaning.

In [13]:
Object_columns = ['hashottuborspa', 'propertycountylandusecode','propertyzoningdesc',
                  'fireplaceflag', 'taxdelinquencyflag','transactiondate']

for column in Object_columns:
    df = combined_df[column].unique()
    print(df)

[nan True]
['010D' '0109' '1200' '1210' '010V' '300V' '0100' '0200' '010C' '0300'
 '1204' '100V' '01HC' '120C' '040V' '1214' '0101' nan '070P' '0700' '1'
 '128' '0' '1112' '5050' '1110' '1118' '010E' '1111' '1129' '1420' '1222'
 '1128' '1310' '1117' '1333' '1116' '1410' '0104' '070D' '0108' '012C'
 '0400' '0103' '01DC' '0201' '010M' '030V' '0110' '010F' '010G' '010H'
 '0301' '0102' '020E' '020M' '0401' '0113' '0141' '122' '38' '135' '96'
 '71' '73' '34' '0209' '040G' '010L' '012E' '0111' '1720' '1014' '012D'
 '1213' '1201' '1321' '1444' '1421' '0140' '0123' '0133' '020G' '030G'
 '105' '0204' '010T' '0120' '1202' '121G' '010X' '020V' '880V' '01HE']
[nan 'LCA11*' 'LAC2' ..., 'SCRM(PD)' 'LBPD13' 'WHM2*']
[nan True]
[nan 'Y']
[nan '2016-01-27' '2016-03-30' '2016-05-27' '2016-06-07' '2016-08-08'
 '2016-08-26' '2016-07-08' '2016-05-11' '2016-06-15' '2016-10-07'
 '2016-05-20' '2016-03-17' '2016-05-18' '2016-04-13' '2016-07-06'
 '2016-12-29' '2016-01-11' '2016-09-07' '2016-05-23' '2016-07-15'


So we have some boolean, catagorical and datetime data. So lets see if the data is recorded as such.

In [14]:
Object_columns_2 = ['hashottuborspa','fireplaceflag', 'taxdelinquencyflag','transactiondate']
for column in Object_columns_2:
    df = combined_df[column].dropna()
    types = []
    for entry in df:
        types.append(type(entry))
    print(set(types))

{<class 'bool'>}
{<class 'bool'>}
{<class 'str'>}
{<class 'str'>}


The two boolean looking columns were in fact boolean. Additional taxdelinquencyflag, 'propertycountylandusecode' and 'propertyzoningdesc' are string based categorical. So we can turn those into int based catagorical.

Finally, we will need to convert transactiondate into datetime.

In [15]:
def str_to_intcat(df,col):
    result = pd.factorize(df[col])[0]
    return pd.Series((v for v in result))

def bool_to_int(df,col):
    result = []
    for entry in df[col]:
        if entry == 'Y' or entry == True:
            result.append(1)
        elif entry == 'N' or entry == False:
            result.append(0)
        else:
            result.append(-1)
    return pd.Series((v for v in result))

def to_datetime(df,col):
    result = []
    for entry in df[col]:
        if pd.isnull(entry) == True:
            result.append(entry)
        else:
            result.append(pd.Timestamp(entry))
    return pd.Series((v for v in result))

Now we will replace the data in combined_df with the new data. 
['hashottuborspa', 'propertycountylandusecode','propertyzoningdesc',
                  'fireplaceflag', 'taxdelinquencyflag','transactiondate']

In [16]:
combined_df['Hot_Tub_or_Spa'] = bool_to_int(combined_df,'hashottuborspa')
combined_df['County_Land_use_Code'] = str_to_intcat(combined_df,'propertycountylandusecode')
combined_df['Zoning_desc'] = str_to_intcat(combined_df,'propertyzoningdesc')
combined_df['Fireplace_flag'] = bool_to_int(combined_df,'fireplaceflag')
combined_df['Tax_Delinquency'] = bool_to_int(combined_df,'taxdelinquencyflag')
combined_df['Transaction_date'] = to_datetime(combined_df, 'transactiondate')

columns_to_drop = ['hashottuborspa', 'propertycountylandusecode','propertyzoningdesc',
                  'fireplaceflag', 'taxdelinquencyflag','transactiondate']

cleaned_df = combined_df.drop(columns_to_drop, 1)
cleaned_df.head()

,Unnamed: 0,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxamount,taxdelinquencyyear,censustractandblock,logerror,Hot_Tub_or_Spa,County_Land_use_Code,Zoning_desc,Fireplace_flag,Tax_Delinquency,Transaction_date
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,0,-1,-1,-1,NaT
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,1,0,-1,-1,NaT
2,2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,20800.37,NaN,NaN,NaN,-1,2,1,-1,-1,NaT
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,...,14557.57,NaN,NaN,NaN,-1,2,1,-1,-1,NaT
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,5725.17,NaN,NaN,NaN,-1,3,2,-1,-1,NaT


To wrap up this notebook I want to note a number of cleaning techniques I have not yet done.

The first is that there are a number columns are nearly empty, which normally I would drop. However, I dont want to do any cleaning that I wouldnt do full data set yet and I do not know which columns are mostly empty on the full data set yet.

The other is a number of statistical cleaning. I would normally try to estimate some of the empty values. Another technique I will try to do at a later date is find out which columns correlate most strongly to the goal which is 'logerror.' Another thing I might try to bin some of the continuous data or some of the larger categories. However, since this update is before the statistics section I will hold off on that for now.

I also want to try using Gamma statistics with the continuous data